In [85]:
def search_state(data, s):
    r = {}
    p_data = population.search(s, False)

    if p_data == None:
        print("No match for geography {0}.".format(s))
        return
    else:
        pass
        #pprint(p_data)
    header = ['Date', 'Cases', 'Deaths']
    c_data = []
    prior_c = 0
    prior_d = 0
    row_count = 0
    for line in data:
        #print(line)
        #break
        row_count += 1
        include = False
        row = line.strip().split(',')
        #print(row)
        if len(row) == 5:
            if row[1].strip().lower() == p_data['STNAME'].strip().lower():
                include = True
                #pint('*' * 60)
            else:
                pass
                #print("{0} != {1}".format(row[1], p_data['STNAME']))
                #print(type(row[1]), type(p_data['STNAME']))
            if include:
                delta = int(row[3]) - prior_c
                delta_d = int(row[4]) - prior_d
                if prior_c > 0:
                    delta_p = float("{:.2f}".format((delta / prior_c) * 100))
                else:
                    delta_p = '-'
                    
                if prior_d > 0:
                    delta_dp = float("{:.4f}".format((delta_d / prior_d) * 100))
                else:
                    delta_dp = '-'
                    
                if delta > -1:
                    c_data.append([row[0], row[3], delta, delta_p, row[4], delta_d, delta_dp])
                prior_c = int(row[3])
                prior_d = int(row[4])
            else:
                pass
                #print("EXCLUDE: ", row)

    #print("Row Count: ", row_count)
    if c_data == []:
        print("No case data for that geography.")
        return
    perc_infected = float("{:.4f}".format(int(c_data[-1][1]) / int(p_data['POPESTIMATE2019']) * 100))
    mortality_rate = float("{:.4f}".format(int(c_data[-1][4]) / int(p_data['POPESTIMATE2019']) * 100))
    case_mortality = float("{:.2f}".format(int(c_data[-1][4]) / int(c_data[-1][1]) * 100))
    #print("Case Rate: ", perc_infected, '%')
    r['case_rate'] = perc_infected
    #print("Mortality (vs. total population): ", mortality_rate, '%')
    r['total_population_mortality_rate'] = mortality_rate
    #print("Case Mortality %: ", case_mortality, '%')
    r['case_mortality_rate'] = case_mortality
    #print("2019 Average deaths/day: ", int(p_data['DEATHS2019']) / 365)
    r['deaths_2019'] = p_data['DEATHS2019']
    #print( "Last 7 day case increase  (%):", float("{:.2f}".format(((int(c_data[-1][1]) - int(c_data[-7][1]))/ int(c_data[-1][1])) * 100 )) )
    r['case_increase_rate_prior_7_days'] = float("{:.2f}".format(((int(c_data[-1][1]) - int(c_data[-7][1]))/ int(c_data[-7][1])) * 100 ))
    
    if int(c_data[-1][4]) > 0:
        #print( "Last 7 day death increase (%):", float("{:.2f}".format(((int(c_data[-1][4]) - int(c_data[-7][4]))/ int(c_data[-1][4])) * 100 )) )
        r['death_increase_rate_prior_7_days'] = float("{:.2f}".format(((int(c_data[-1][4]) - int(c_data[-7][4]))/ int(c_data[-7][4])) * 100 ))

        
    else:
        r['death_increase_rate_prior_7_days'] = 0
    #print(tabulate(c_data, headers=['Date', 'Cases', 'Delta', '% Inc', 'Deaths', 'Delta', '% Inc']))
    r['chronological_data'] = c_data
    r['chronological_data_table'] = tabulate(c_data, headers=['Date', 'Cases', 'Delta', '% Inc', 'Deaths', 'Delta', '% Inc'])
    r['demographic_data'] = p_data
    r['state'] = s
    r['FIPS'] = p_data['composite_fips']
    r['total deaths'] = int(c_data[-1][4])
    r['total cases'] = int(c_data[-1][1])
    return r

In [86]:
#state_data = pd.read_csv("./data/CO-EST2019-alldata.csv")
#state_df = pd.DataFrame(dem_data, columns = [ 'STATE', 'COUNTY', 'POPESTIMATE2019','DEATHS2019'])
    
#dem_df['FIPS'] = dem_df.apply(fips_glue, axis = 1)


In [87]:
#import search_geography
from pprint import pprint
from tabulate import tabulate
import pandas as pd

import population

from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))
#county_data = open("../covid-19-data/us-counties.csv")
#pprint(search_geography.search_county(county_data, 'iowa', 'polk'))
state_data = open("../covid-19-data/us-states.csv")
state_list = open("./data/CO-EST2019-alldata.csv")
results = []
for line in state_list:
    row = line.split(',')
    if row[0] == '040':
        #print(row[0], row[3], row[5])
        state_data = open("../covid-19-data/us-states.csv")
        foo = search_state(state_data, row[5].strip())
#print(foo['chronological_data_table'])
        #print(row[5])
        if foo:
            #print( "Last 7 day case increase (%):", foo['case_increase_rate_prior_7_days'])
            #print( "Last 7 day death increase (%):", foo['death_increase_rate_prior_7_days'])
            #print('-' * 80
            result = [row[5], foo['case_increase_rate_prior_7_days'], foo['death_increase_rate_prior_7_days'], foo['case_rate'],
                     foo['case_mortality_rate'], foo['total cases'], foo['total deaths']]
            results.append(result)
#print(search_state(state_data, 'Alaska'))
#print(results)
df3 = pd.DataFrame(results, columns=['State', '7 day case%', '7 day death%', 'case rate', 'case mortality', 'total cases', 'total deaths'])

df3.sort_values(by=['7 day case%'], ascending=False, inplace=True)
display(df3)

,State,7 day case%,7 day death%,case rate,case mortality,total cases,total deaths
26,Montana,40.75,26.09,0.1645,1.65,1758,29
12,Idaho,33.43,8.51,0.6139,0.93,10971,102
9,Florida,30.69,12.28,1.2562,1.57,269803,4241
43,Texas,26.49,19.08,0.9133,1.23,264811,3246
1,Alaska,26.48,7.14,0.2422,0.85,1772,15
36,Oklahoma,23.67,5.76,0.5114,2.09,20235,422
48,West Virginia,23.30,1.05,0.2368,2.26,4244,96
40,South Carolina,22.14,16.20,1.1002,1.70,56648,961
28,Nevada,21.00,10.43,0.9008,2.14,27745,593
2,Arizona,20.62,22.22,1.6827,1.83,122479,2239
